<a href="https://colab.research.google.com/github/YorkeTripleThree/Chatbot/blob/main/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Students run this first - takes about 1-2 minutes
print("🤖 Welcome to AI Chatbot Builder for Business!")
print("=" * 50)
print("Installing required packages... This may take 1-2 minutes ⏳")

!pip install transformers torch gradio accelerate -q

# Import all required libraries
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import gradio as gr
import json
import warnings
import re
warnings.filterwarnings('ignore')

print("Setup complete! Ready to build your chatbot.")
print("Next: Run Cell 2 to create your business knowledge base")


In [ ]:
print("Creating Your Business Knowledge Base")

knowledge_base = {
    "business_name": "Pages & Chapters Bookstore",
    "business_type": "Bookstore",
    "about": "A cozy bookstore offering new and used books, reading events, and a quiet café with coffee and pastries",
    "products_or_services": [
        "New books (fiction, non-fiction, young adult)",
        "Used books at discounted prices",
        "E-books available online",
        "Book club events and author signings",
        "Gift cards and bookstore merchandise",
        "Coffee and pastries at our in-store café"
    ],
    "common_questions": {
        "What are your hours?": "We're open Monday-Saturday 9 AM - 7 PM, Sunday 10 AM - 5 PM",
        "Where are you located?": "We're at 456 Book Lane, near the city library",
        "Do you sell gift cards?": "Yes, we offer gift cards in any amount!",
        "Can I order online?": "Absolutely, visit our website at www.pagesandchapters.com!",
        "Do you have books for kids?": "We have a great selection of children's books in-store and online!",
        "Do you host events?": "Yes, we host book clubs and author signings monthly. Check our website for the schedule!",
        "What else can you help with?": "I can answer questions about our books, events, hours, or online orders. What's on your mind?",
        "Can I return a book?": "Yes, we accept returns within 30 days with a receipt. See our website for details.",
        "Do you have textbooks?": "We carry a selection of textbooks, especially for local university courses.",
        "What if I ask something unrelated?": "I'm here to help with bookstore questions! Try asking about our books, events, or café, or email us at hello@pagesandchapters.com."
    },
    "contact_info": {
        "phone": "555-READ-456",
        "email": "hello@pagesandchapters.com",
        "website": "www.pagesandchapters.com"
    },
    "personality": {
        "tone": "warm and bookish",
        "style": "casual but knowledgeable"
    }
}

print(f"Knowledge base created for {knowledge_base['business_name']} ({knowledge_base['business_type']})!")
print(f"Business Description: {knowledge_base['about']}")
print(f"Products/Services: {', '.join(knowledge_base['products_or_services'])}")
print(f"Contact: {knowledge_base['contact_info']['phone']}, {knowledge_base['contact_info']['email']}")


In [ ]:
print("Loading AI Model...")

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

try:
    model_name = "microsoft/DialoGPT-small"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)
    # Configure model to reduce randomness in responses
    model.config.temperature = 0.7  # Lower temperature for more focused responses
    model.config.top_k = 40  # Top-k sampling to limit to top 40 words
    model.config.pad_token_id = tokenizer.eos_token_id  # Proper padding
    print(f"AI model {model_name} loaded successfully!")
except Exception as e:
    print(f"⚠️ Failed to load AI model: {e}")
    print("Falling back to knowledge base mode (no AI responses)")
    model = None
    tokenizer = None

print("Ready to create chatbot logic in the next cell!")


In [ ]:
# Replace Cell 4 with this improved version

print("🤖 Creating Enhanced Chatbot Response Logic...")

import random
import re

def enhanced_chatbot_response(query):
    """
    Enhanced chatbot response function with better off-topic detection
    """
    if not query or not query.strip():
        return "Please ask me a question about our business!"

    query_lower = query.lower().strip()

    # Greeting detection
    greetings = ["hello", "hi", "hey", "good morning", "good afternoon", "good evening"]
    if any(greeting in query_lower for greeting in greetings) and len(query_lower.split()) <= 3:
        return f"Hello! Welcome to {knowledge_base['business_name']}! How can I help you today?"

    # Direct question matching from knowledge base
    for question, answer in knowledge_base["common_questions"].items():
        # Check for exact matches or key phrases
        question_words = question.lower().split()
        if any(word in query_lower for word in question_words if len(word) > 3):
            # Additional validation for better matching
            if "hours" in question.lower() and "hours" in query_lower:
                return answer
            elif "location" in question.lower() and any(word in query_lower for word in ["where", "location", "address"]):
                return answer
            elif "gift" in question.lower() and "gift" in query_lower:
                return answer
            elif "online" in question.lower() and "online" in query_lower:
                return answer
            elif "kids" in question.lower() and any(word in query_lower for word in ["kids", "children", "child"]):
                return answer
            elif "events" in question.lower() and "events" in query_lower:
                return answer
            elif "return" in question.lower() and "return" in query_lower:
                return answer
            elif "textbook" in question.lower() and "textbook" in query_lower:
                return answer

    # Enhanced keyword detection with better context
    business_keywords = [
        "hours", "open", "close", "time", "schedule", "operating",
        "location", "where", "address", "directions", "find",
        "books", "products", "services", "offer", "sell", "available",
        "events", "coffee", "café", "gift", "cards", "online",
        "contact", "phone", "email", "website"
    ]

    # Off-topic detection - things clearly not related to a bookstore
    off_topic_keywords = [
        "weather", "temperature", "rain", "snow", "climate",
        "sports", "football", "basketball", "soccer", "game",
        "politics", "election", "president", "government", "vote",
        "news", "breaking", "headlines", "current events",
        "movie", "film", "cinema", "actor", "actress",
        "music", "song", "album", "artist", "concert",
        "food", "recipe", "cooking", "restaurant", "eat",
        "travel", "vacation", "flight", "hotel", "trip",
        "health", "doctor", "medicine", "hospital", "sick",
        "technology", "computer", "software", "programming", "code",
        "fashion", "clothes", "shopping", "brand", "style",
        "celebrities", "famous", "star", "hollywood"
    ]

    # Check if the query is clearly off-topic
    if any(keyword in query_lower for keyword in off_topic_keywords):
        return f"I'm here to help with questions about {knowledge_base['business_name']}! I can tell you about our books, events, hours, location, or café. What would you like to know?"

    # Business-related responses
    if any(keyword in query_lower for keyword in ["hours", "open", "close", "time"]):
        return knowledge_base["common_questions"]["What are your hours?"]

    elif any(keyword in query_lower for keyword in ["where", "location", "address", "directions"]):
        return knowledge_base["common_questions"]["Where are you located?"]

    elif any(keyword in query_lower for keyword in ["contact", "phone", "email", "reach"]):
        return f"Here's how to reach us:\n📞 Phone: {knowledge_base['contact_info']['phone']}\n📧 Email: {knowledge_base['contact_info']['email']}\n🌐 Website: {knowledge_base['contact_info']['website']}"

    elif any(keyword in query_lower for keyword in ["products", "services", "offer", "sell", "books"]):
        return f"We offer: {', '.join(knowledge_base['products_or_services'])}. What specific type of book or service interests you?"

    elif any(keyword in query_lower for keyword in ["about", "tell me", "what is", "describe"]):
        return f"We're {knowledge_base['about']}! We offer {', '.join(knowledge_base['products_or_services'][:3])} and more. What would you like to know specifically?"

    # If query contains business keywords but no specific match
    elif any(keyword in query_lower for keyword in business_keywords):
        return f"I'd be happy to help with that! I can provide information about our books, events, hours, location, or café services. Could you be more specific about what you'd like to know about {knowledge_base['business_name']}?"

    # For unclear or unrelated queries
    else:
        return f"I'm the AI assistant for {knowledge_base['business_name']}, and I'm here to help with questions about our bookstore! I can tell you about our books, events, hours, location, or café. What would you like to know?"

def get_conversation_starters():
    """
    Generate example questions for the interface
    """
    return [
        "What are your hours?",
        "Where are you located?",
        f"What does {knowledge_base['business_name']} offer?",
        "Do you have events?",
        "Can I order online?",
        "Do you have gift cards?"
    ]

print("✅ Enhanced chatbot logic created!")
print("Features:")
print("• Better off-topic detection")
print("• Improved keyword matching")
print("• Context-aware responses")
print("• Graceful handling of unclear queries")
print("• Professional fallback responses")
print("Ready to create the web interface!")

In [ ]:
# This creates the user interface for your chatbot
print("Creating Web Interface...")
print("=" * 35)

def chat_interface(message):
    """
    Simple chat interface function for basic Gradio Interface
    """
    if not message.strip():
        return "Please ask me a question about our business!"

    response = enhanced_chatbot_response(message)
    return response

# Create conversation examples
examples = get_conversation_starters()

# Create the Gradio interface
print("Building interface components...")

def simple_chat_interface(message):
    """Simple chat interface function"""
    if not message.strip():
        return "Please ask me a question about our business!"

    response = enhanced_chatbot_response(message)
    return response

# Create a simple interface that works with all Gradio versions
interface = gr.Interface(
    fn=simple_chat_interface,
    inputs=gr.Textbox(
        label="Ask me about our business:",
        placeholder="e.g., What are your hours? What do you offer?",
        lines=2
    ),
    outputs=gr.Textbox(
        label="Chatbot Response:",
        lines=5
    ),
    title=f" {knowledge_base['business_name']} - AI Assistant",
    description=f"Hi! I'm the AI assistant for **{knowledge_base['business_name']}**.\n\n{knowledge_base['about']}\n\nAsk me anything about our business!",
    examples=examples,
    theme="default"
)

print("✅ Interface created successfully!")
print(f"Title: {knowledge_base['business_name']} - AI Assistant")
print("Next: Run Cell 6 to launch your chatbot!")

In [ ]:
print(" Launching Your Chatbot!")
print("=" * 35)
print("Starting web interface... This will create a link you can click to test your chatbot!")

try:
    # Launch the interface
    interface.launch(
        share=True,  # Creates a temporary public link
        debug=False,
        show_error=True
    )

    print("✅ Chatbot launched successfully!")
    print("\n" + "="*50)
    print("CONGRATULATIONS! Your chatbot is ready!")
    print("="*50)
    print("Click the link above to test your chatbot")
    print("Try asking:")
    for example in examples[:3]:
        print(f"  • {example}")
    print("\nTips for testing:")
    print("  • Try questions from your knowledge base")
    print("  • Ask questions NOT in your knowledge base")
    print("  • Test with spelling mistakes")
    print("  • Try conversational phrases")

except Exception as e:
    print(f"⚠️ Launch error: {e}")
    print("Don't worry! You can still test the chatbot logic manually.")

In [ ]:
# Students can use this to test their chatbot manually
print("\n Manual Testing Section")
print("=" * 30)
print("If the web interface didn't work, you can test your chatbot here:")

def test_chatbot():
    """
    Function to manually test the chatbot
    """
    print(f"\nTesting {knowledge_base['business_name']} Chatbot:")
    print("-" * 40)

    test_questions = [
        "Hello!",
        "What are your hours?",
        f"Tell me about {knowledge_base['business_name']}",
        "What do you offer?",
        "How can I contact you?",
        "Do you have discounts?",
        "This is a random question not in your knowledge base"
    ]

    for question in test_questions:
        print(f"\n Question: {question}")
        response = enhanced_chatbot_response(question)
        print(f" Response: {response}")
        print("-" * 40)

# Uncomment the line below to run manual testing
test_chatbot()

print("Manual testing function ready!")
print("Uncomment the last line in this cell to run automatic tests")


In [ ]:
# This helps students understand what they've built
print(" Chatbot Analysis & Learning")
print("=" * 35)

def analyze_chatbot():
    """
    Analyze the chatbot's capabilities and limitations
    """
    print(f"Analysis of {knowledge_base['business_name']} Chatbot:")
    print("=" * 50)

    print("\n STRENGTHS:")
    print("• Can answer questions about business hours")
    print("• Provides contact information")
    print("• Lists products and services")
    print("• Handles common customer questions")
    print("• Maintains consistent brand voice")
    print("• Responds to greetings appropriately")

    kb_questions = len(knowledge_base["common_questions"])
    products = len(knowledge_base["products_or_services"])

    print(f"• Has {kb_questions} pre-programmed answers")
    print(f"• Knows about {products} products/services")

    print("\n LIMITATIONS:")
    print("• Cannot handle complex, multi-part questions")
    print("• Cannot process orders or make transactions")
    print("• Cannot access real-time information (inventory, etc.)")
    print("• May not understand slang or very casual language")
    print("• Cannot learn from conversations (no memory)")
    print("• Limited creativity for questions outside knowledge base")

    print("\n BUSINESS VALUE:")
    print("• Could handle 24/7 basic customer inquiries")
    print("• Reduces staff time spent on repetitive questions")
    print("• Provides consistent brand messaging")
    print("• Could collect customer contact information")
    print("• Scalable - can handle multiple customers simultaneously")

    print("\n POTENTIAL IMPROVEMENTS:")
    print("• Add more detailed product information")
    print("• Include pricing information")
    print("• Add appointment scheduling capability")
    print("• Connect to inventory systems")
    print("• Add multi-language support")
    print("• Include sentiment analysis")

    print("\nREAL-WORLD IMPLEMENTATION NEEDS:")
    print("• Professional hosting and deployment")
    print("• Integration with business systems")
    print("• Regular knowledge base updates")
    print("• Customer feedback and improvement cycles")
    print("• Compliance with data privacy regulations")
    print("• Backup to human agents for complex issues")

# Run the analysis
analyze_chatbot()

print("\n" + "="*50)
print("LEARNING QUESTIONS:")
print("="*50)
print("1. What types of questions does your chatbot handle well?")
print("2. What questions does it struggle with?")
print("3. How could this chatbot benefit a real business?")
print("4. What would you need to add to make it production-ready?")
print("5. What surprised you about AI capabilities or limitations?")
print("6. How might customers react to chatting with an AI?")
print("7. What ethical considerations should businesses think about?")


In [ ]:
# Students can modify and experiment
print(" Customization Playground")
print("=" * 30)
print("Students: Try modifying the code above!")

# Example customizations students can try:
customization_ideas = {
    "personality": "Change the chatbot's tone (formal, casual, funny, professional)",
    "knowledge_base": "Add more questions and answers to the common_questions section",
    "products": "Add detailed product descriptions with prices",
    "features": "Add new keyword detection for different types of questions",
    "responses": "Make responses more creative or detailed",
    "business_type": "Create a completely different business scenario"
}

print("\n Customization Ideas:")
for feature, description in customization_ideas.items():
    print(f"• {feature.title()}: {description}")

print("\n To implement changes:")
print("1. Modify the knowledge_base in Cell 2")
print("2. Adjust the response logic in Cell 4 if needed")
print("3. Re-run cells 5-6 to see your changes")

print("\n Experimentation setup complete!")

In [ ]:
# FINAL SUMMARY
print("\n" + "/" * 20)
print("CONGRATULATIONS! YOU'VE BUILT AN AI CHATBOT!")
print("/" * 20)
print(f"\nYour {knowledge_base['business_name']} chatbot includes:")
print(" Custom business knowledge base")
print(" AI-powered conversation capability")
print(" Web-based chat interface")
print(" Intelligent response matching")
print(" Professional business personality")

print(f"\n Business Impact Potential:")
print("• Could handle basic customer service 24/7")
print("• Reduces repetitive work for human staff")
print("• Provides consistent brand experience")
print("• Scalable to handle multiple customers")

print(f"\n Skills Learned:")
print("• AI integration for business applications")
print("• Knowledge base design and structure")
print("• Understanding AI capabilities and limitations")
print("• Practical experience with chatbot technology")

print("\n" + "="*50)
print("Ready to show off your chatbot! ")
print("="*50)